# Decision Tree

**Importing dependencies :**

In [6]:
import numpy as numpy
import math
import csv

**Class definition :**

In [7]:
class Node:
    def __init__(self,attribute):
        self.attribute = attribute
        self.children = []
        self.answer = None

**Method definitions :**

In [8]:
def read_data(filename):
    with open(filename,'r') as csvFile:
        dReader = csv.reader(csvFile,delimiter=',')
        headers = next(dReader)
        dTrain = [row for row in dReader]
    return headers, dTrain

In [9]:
def read_test_data(filename):
    with open(filename,'r') as csvFile:
        dReader = csv.reader(csvFile,delimiter=',')
        dTest = [row for row in dReader]
    return dTest